# The Battle of the Neighborhoods: San Francisco

As a reminder, my goal is to find the most desirable neighborhood for renters in San Francisco. I will make a suggestion by finding the neighborhoods will the cheapest average monthly, and then comparing the types and amount of recommended venues in the neighborhoods. Additionally, I am including the more expensive neighborhoods in my analysis to see how well the amenities of the cheaper and more expensive neighborhoods align.

I will begin by importing the necessary libraries.

In [1]:
# To import Numpy and Pandas libraries
!pip install numpy
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# To import library to handle JSON files
import json

# To convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim 

# To handle requests
import requests

#To import libraries necessary for web scraping
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

# To transform JSON into a pandas dataframe
from pandas import json_normalize

# Plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# To import k-means from clustering stage
from sklearn.cluster import KMeans

# To import library to render maps
!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


I will then web scrape from RENTCafé the table of average rent by neighborhood in San Francisco, CA.

In [2]:
url = 'https://www.rentcafe.com/average-rent-market-trends/us/ca/san-francisco/'

In [3]:
req = Request('https://www.rentcafe.com/average-rent-market-trends/us/ca/san-francisco/', headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req).read()

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
tables = soup.find_all('table', id = 'MarketTrendsAverageRentTable')

In [6]:
Neighborhood = []
AverageRent = []

for table in tables:
    rows = table.find_all('tr')
    
    for row in rows:
        neighborhood = row.find_all('th')
        averagerent = row.find_all('td')
         
        for i in range(0, len(averagerent)):
            Neighborhood.append(neighborhood[i].text.strip())
            AverageRent.append(averagerent[i].text.strip())
        # Both need to be len(averagerent) because len(neighborhood) inludes the headers

In [7]:
df = pd.DataFrame(list(zip(Neighborhood, AverageRent)), columns = ['Neighborhood', 'Average Rent'])

In [8]:
df

,Neighborhood,Average Rent
0,Treasure Island,"$2,616"
1,Van Ness - Civic Center,"$2,944"
2,Tenderloin,"$2,944"
3,Downtown District 8 - North East,"$2,956"
4,Marina,"$2,974"
5,Hayes Valley,"$2,983"
6,Russian Hill,"$2,983"
7,Nob Hill,"$2,996"
8,Western Addition,"$3,030"
9,Bernal Heights,"$3,061"


Conveniently, RENTCafé already sorts the neighborhoods by average rent, starting with the lowest values. As you can see, __Treasure Island__ is the neighborhood with the cheapest average monthly rent, followed by __Van Ness - Civic Center, Tenderloin, Downtown District 8 - North East, and Marina.__ I will be keeping an eye on these neighborhoods, as they are the five cheapest.

In order to visualize a map of the neighborhoods and later use the Foursquare API, I will need the geographical coordinates for each of the neighborhoods. This can be easily accomplished with the geopy library's geolocation features.

Before doing so, I need the coordinates for San Francisco as a whole.

In [9]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent = 'sf_explorer')

location = geolocator.geocode(address)
sf_lat = location.latitude
sf_lon = location.longitude

print('The geographical coordinates of San Francisco are {}, {}.'.format(sf_lat, sf_lon))

The geographical coordinates of San Francisco are 37.7790262, -122.4199061.



Next, I will get the coordinates for each of the neighborhoods. I have to make a few small changes to the neighborhoods to get the correct coordinates: 

In the cells below, I will create a copy of the original dataframe and replace some of the neighborhoods with either their more commonly used names, or with a landmark to help distinguish them from other California regions of the same name. After getting the coordinates, I will change the neighborhoods back to their original names for clarity.

Additionally, in the cell below that, I drop "Central Sunset," which is a smaller part of "Sunset District." Central Sunset returns the exact same coordinates as Sunset District and has the same average rent, and so it acts as a duplicate and is not relevant.

All this data cleaning must be performed so the geolocator knows the regions to which I'm referring and does not return incorrect coordinates.

In [10]:
df = df[~df['Neighborhood'].str.contains('Central Sunset')].reset_index(drop = True)

I am making a copy of the dataframe so I can change the names of the neighborhoods in a temporary dataframe. Once I have the coordinates, I will be able to concatenate them with the original df so as to retain the oriignal names.

In [11]:
copy_df = df.copy()

In [12]:
copy_df['Neighborhood'].replace({'Downtown District 8 - North East': 'Union Square', 
                                  'Marina': 'Marina District',
                                  'Bayview': 'Bayview District',
                                  'Financial District - Barbary Coast': 'Financial District',
                                  'Golden Gate Heights': 'Golden Gate Heights Park', 
                                  'Outer Sunset': 'Sunset District', 
                                  'Hunters Point': 'Hunters Point, San Francisco', 
                                  'North Waterfront': 'Northeast Waterfront Historic District', 
                                  'Forest Knolls': 'Mt Sutro Open Space Reserve', 
                                  'Inner Mission': 'Mission District',
                                  'Merced Manor': 'Merced Manor Reservoir',
                                  'Lakeside': 'Mercy High School, San Francisco',
                                  'Lake Shore': 'Lake Merced Park',
                                  'Ingleside Heights': 'Randolph Street & Arch Street',
                                  'Sunnyside': 'Sunnyside, San Francisco',
                                  'Lake Street': 'Mountain Lake Park',
                                  'Jordan Park - Laurel Heights': 'Jordan Park',
                                  'Inner Richmond': 'Inner Richmond District',
                                  'Central Richmond': 'Central Richmond District',
                                  'Parnassus Heights': 'Parnassus Avenue',
                                  'Bayview Heights': 'Le Conte',
                                  'Buena Vista': 'Buena Vista Park, San Francisco',
                                  'Ashbury Heights': 'Mt Olympus, San Francisco',
                                  'Portola': 'Portola, San Francisco',
                                  'Little Hollywood': 'Little Hollywood Park',
                                  'Westwood Park': 'Westwood Drive, San Francisco',
                                  'Outer Mission': 'Cayuga Park',
                                  'Mount Davidson Manor': 'Aptos Middle School',
                                  'Mission Terrace': 'Balboa Park, San Francisco',
                                  'Yerba Buena': 'Yerba Buena Gardens',
                                  'South Beach': 'South Beach, San Francisco',
                                  'Eureka Valley - Dolores Heights': 'Dolores Heights', 
                                  'Midtown Terrace': 'Sutro Reservoir'}, inplace = True)

In [13]:
copy_df.head()

,Neighborhood,Average Rent
0,Treasure Island,"$2,616"
1,Van Ness - Civic Center,"$2,944"
2,Tenderloin,"$2,944"
3,Union Square,"$2,956"
4,Marina District,"$2,974"


After fixing the neighborhood names, I can run a for loop to get the correct coordinates for each neighborhood.

In [15]:
lat = []
lon = []

for i in copy_df['Neighborhood']:
    geolocator = Nominatim(user_agent = 'sf_explorer')
    
    address = (i + ', California')
    
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude

    lat.append(latitude)
    lon.append(longitude)

In [16]:
loc_df = pd.DataFrame(list(zip(lat, lon)), columns=['Latitude', 'Longitude'])
loc_df

,Latitude,Longitude
0,37.823354,-122.370153
1,37.775190,-122.419266
2,37.784249,-122.413993
3,37.787936,-122.407517
4,37.802984,-122.437472
5,37.776685,-122.422936
6,37.800073,-122.417094
7,37.793262,-122.415249
8,37.779559,-122.429810
9,37.742986,-122.415804


Now, I can concatenate the location dataframe with the original df to get the full data. As mentioned, this also brings back the correct neighborhood names.

In [17]:
full_df = pd.concat([df, loc_df], axis = 1)
full_df.head()

,Neighborhood,Average Rent,Latitude,Longitude
0,Treasure Island,"$2,616",37.823354,-122.370153
1,Van Ness - Civic Center,"$2,944",37.775190,-122.419266
2,Tenderloin,"$2,944",37.784249,-122.413993
3,Downtown District 8 - North East,"$2,956",37.787936,-122.407517
4,Marina,"$2,974",37.802984,-122.437472


With this full dataframe, I can now visualize the neighborhoods on a map. I will use the folium library to do so.

In [18]:
sf_map = folium.Map(location = [sf_lat, sf_lon], zoom_start = 12)

for lat, lon, neighborhood in zip(full_df['Latitude'], full_df['Longitude'], full_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 7,
        popup = label,
        color = 'darkblue',
        fill = True,
        fill_color = '#AA789A',
        fill_opacity = 0.9,
        parse_html = False).add_to(sf_map)
    
sf_map

Something interesting to note here is that, for the most part, the cheapest neighborhoods tend to be in the northeast section of the city.

Now that I have my map with the cheapest neighborhoods plotted, I can get to work with the Foursquare API to find the recommended venues in the area. In the hidden cell below, I define my credentials and version type, which will be used to generate the Foursquare API URL.

In [19]:
# @hidden_cell
CLIENT_ID = 'XUKV0ERQ5N5GHGT4IL4ZY4OQXFV1KK5IQV4WAI5CEQMR25JB' # your Foursquare ID
CLIENT_SECRET = 'E5TQFWQGWKD5S4WQOFZI5B1WAP0TXEXBBPXMF1LP51KXAC1W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

I'll explore the recommended venues surrounding the first neighborhood in the dataframe (Treasure Island), and then I will repeat the analysis with each neighborhood. First, I'll need to define the latitude and longitude for the neighborhood.

In [20]:
neighborhood_latitude = full_df.loc[0, 'Latitude']
neighborhood_longitude = full_df.loc[0, 'Longitude']

I will create the GET request URL to get the top 100 venues in Treasure Island in a 500-meter radius.

In [21]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=XUKV0ERQ5N5GHGT4IL4ZY4OQXFV1KK5IQV4WAI5CEQMR25JB&client_secret=E5TQFWQGWKD5S4WQOFZI5B1WAP0TXEXBBPXMF1LP51KXAC1W&v=20180605&ll=37.8233541,-122.37015310228705&radius=500&limit=100'

This URL will return the data for Treasure Island, as shown below. (Note that this output is quite long, and may take a few seconds to scroll through.

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb6cb75d91d7d1d496783da'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Treasure Island',
  'headerFullLocation': 'Treasure Island, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 37.827854104500005,
    'lng': -122.364466846621},
   'sw': {'lat': 37.8188540955, 'lng': -122.3758393579531}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4495432ef964a52083341fe3',
       'name': 'Treasure Island',
       'location': {'crossStreet': 'at California Ave',
        'lat': 37.824090595823414,
        'lng': -122.37070083618163,
        'distance': 95,
        'postalCode': '9

Now, I will define a function to extract the category of a venue.

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

The resulting JSON can now be cleaned and structured into a dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Treasure Island,Island,37.824091,-122.370701
1,Mersea,American Restaurant,37.822473,-122.375594
2,Treasure Island Flea,Flea Market,37.821020,-122.374469
3,Tepper Field,Baseball Field,37.823249,-122.364783
4,Island Cove Market,Grocery Store,37.819756,-122.372041


In [25]:
print('{} venues were returned by Foursquare for Treasure Island.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare for Treasure Island.


Now, I can repeat this process on each neighborhood in the dataframe.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
sf_venues = getNearbyVenues(names = full_df['Neighborhood'], latitudes = full_df['Latitude'], longitudes = full_df['Longitude'])

Treasure Island
Van Ness - Civic Center
Tenderloin
Downtown District 8 - North East
Marina
Hayes Valley
Russian Hill
Nob Hill
Western Addition
Bernal Heights
Lower Pacific Heights
Pacific Heights
Telegraph Hill
North Beach
Noe Valley
Silver Terrace
Hunters Point
Candlestick Point
Bayview Heights
Bayview
Financial District - Barbary Coast
Outer Sunset
Inner Sunset
Golden Gate Heights
North Panhandle
Alamo Square
Anza Vista
North Waterfront
Midtown Terrace
Glen Park
Diamond Heights
Cow Hollow
Forest Knolls
Inner Mission
Stonestown
Pine Lake Park
Merced Manor
Merced Heights
Lakeside
Lake Shore
Ingleside Heights
Mission Dolores
Sunnyside
Presidio Heights
Lake Street
Jordan Park - Laurel Heights
Inner Richmond
Central Richmond
Lone Mountain
Golden Gate Park
Parnassus Heights
Haight Ashbury
Cole Valley
Buena Vista
Ashbury Heights
Visitacion Valley
Portola
Little Hollywood
South of Market
Potrero Hill
Twin Peaks
John McLaren Park
Crocker Amazon
Westwood Park
Outer Mission
Oceanview
Mount Davi

I will see how many venues were returned for each neighborhood.

In [28]:
sf_venues.groupby(['Neighborhood'])['Venue'].count().sort_values(ascending = False)

Neighborhood
Yerba Buena                           100
Cow Hollow                            100
Downtown District 8 - North East      100
Tenderloin                            100
Sunnyside                             100
Hayes Valley                          100
South Beach                           100
Inner Mission                         100
Marina                                100
Telegraph Hill                         99
Haight Ashbury                         95
North Beach                            93
South of Market                        92
Van Ness - Civic Center                91
North Panhandle                        88
Financial District - Barbary Coast     77
Alamo Square                           77
North Waterfront                       74
Bernal Heights                         73
Duboce Triangle                        71
Nob Hill                               63
Mission Bay                            62
Lower Pacific Heights                  62
Pacific Heights      

Although Treasure Island is the cheapest neighborhood overall, it is one of the neighborhoods with the fewest venues. Meanwhile, four of the neighborhoods that hit the 100 venue limit (Tenderloin, Union Square, Marina District, and Hayes Valley) are 3rd-6th place on the list of cheapest neighborhoods, with __Tenderloin__ being the cheapest of the four.

Next, I will look at the number of *unique* categories in each neighborhood. This will help me to ascertain which neighborhoods have the most diverse venues. It it important to understand that by virtue of some neighborhoods having fewer venues overall, they will also have less diversity of venues.

In [29]:
sf_venues.groupby(['Neighborhood'])['Venue Category'].nunique().sort_values(ascending = False)

Neighborhood
Sunnyside                             68
Inner Mission                         68
Cow Hollow                            66
Marina                                66
South of Market                       65
Hayes Valley                          63
Yerba Buena                           62
Alamo Square                          62
Van Ness - Civic Center               60
Telegraph Hill                        59
Downtown District 8 - North East      57
South Beach                           55
North Beach                           53
North Panhandle                       53
Duboce Triangle                       53
Haight Ashbury                        53
Bernal Heights                        51
Tenderloin                            51
Financial District - Barbary Coast    49
Lower Pacific Heights                 48
North Waterfront                      47
Pacific Heights                       44
Noe Valley                            43
Cole Valley                           42
Jor

**Sunnyside** and **Inner Mission** have the most unique categories out of all the neighborhoods. However, this does not necessarily make it the most diverse, as other neighborhoods are close behind in the number of unique venues they have. Furthermore, some of the venue categories in Foursquare are closely related. 

For example, "Bar" and "Cocktail Bar" or "Gym" and "Gym / Fitness Center" are all different categories in the API. However, each pair contains almost identical business types. Having both a bar and a cocktail bar in a neighborhood would not mean that it has diverse venues, nor would having both a gym and a gym/fitness center. Because of this, I want to examine the top most common venues in each neighborhood.

I will start by using the one-hot encoding technique, which converts categorical variables to numerical ones.

In [31]:
sf_dummies = pd.get_dummies(sf_venues[['Venue Category']], prefix = "", prefix_sep = "")

In [39]:
sf_dummies.columns[205]

'Neighborhood'

In [40]:
sf_dummies = pd.get_dummies(sf_venues[['Venue Category']], prefix = "", prefix_sep = "")

sf_dummies.rename(columns={sf_dummies.columns[205]: 'Neighborhood Venue' }, inplace = True)

sf_dummies['Neighborhood'] = sf_venues['Neighborhood']

fixed_columns = [sf_dummies.columns[-1]] + list(sf_dummies.columns[:-1])
sf_dummies = sf_dummies[fixed_columns]

sf_dummies.head()

,Neighborhood,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,Neighborhood Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Physical Therapist,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Rugby Pitch,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Skate Park,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spa

Using the pandas ```groupby``` function, I can get the average frequency of occurrence for each category.

In [41]:
sf_avg = sf_dummies.groupby('Neighborhood').mean().reset_index()
sf_avg

,Neighborhood,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,College Gym,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Mountain,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nabe Restaurant,Nail Salon,Neighborhood Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Physical Therapist,Pier,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Rugby Pitch,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Skate Park,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spa

Now that I have the frequency of each category for each neighborhood, I can look at the most common venue types in each neighborhood sorted by their frequency.

For the sake of saving you the need to scroll, I'll limit the number of venues to 5.

In [67]:
num_top_venues = 5

for hood in sf_avg['Neighborhood']:
    print("----"+hood+"----")
    temp = sf_avg[sf_avg['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Alamo Square----
          venue  freq
0           Bar  0.05
1   Pizza Place  0.03
2   Record Shop  0.03
3      Wine Bar  0.03
4  Liquor Store  0.03


----Anza Vista----
            venue  freq
0            Café  0.15
1     Coffee Shop  0.10
2      Donut Shop  0.05
3  Cosmetics Shop  0.05
4   Grocery Store  0.05


----Ashbury Heights----
            venue  freq
0  Breakfast Spot  0.08
1            Park  0.08
2             Gym  0.05
3  Cosmetics Shop  0.05
4        Wine Bar  0.05


----Bayview----
                             venue  freq
0  Southern / Soul Food Restaurant  0.17
1               Light Rail Station  0.08
2               Mexican Restaurant  0.08
3                             Café  0.08
4                           Bakery  0.08


----Bayview Heights----
                       venue  freq
0               Burger Joint  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                       Park  0.25
4         Persian Restaurant  0.00


----Bernal

For viewing ease, I'll put the most common venues for each neighborhood in a pandas dataframe. This time, I will include the 10 most common venues as opposed to only 5. Before doing so, I'll add in a function that sorts the venues from most common to least.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

sorted_venues = pd.DataFrame(columns = columns)
sorted_venues['Neighborhood'] = sf_avg['Neighborhood']

for ind in np.arange(sf_avg.shape[0]):
    sorted_venues.iloc[ind, 1:] = return_most_common_venues(sf_avg.iloc[ind, :], num_top_venues)

sorted_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alamo Square,Bar,Pizza Place,Café,Sushi Restaurant,Seafood Restaurant,Indian Restaurant,Wine Bar,Ethiopian Restaurant,Liquor Store,Record Shop
1,Anza Vista,Café,Coffee Shop,Health Food Store,Eye Doctor,Grocery Store,Southern / Soul Food Restaurant,Health & Beauty Service,Arts & Crafts Store,Sandwich Place,Cosmetics Shop
2,Ashbury Heights,Breakfast Spot,Park,Coffee Shop,Wine Bar,Convenience Store,Cosmetics Shop,Gym,Middle Eastern Restaurant,Trail,Bar
3,Bayview,Southern / Soul Food Restaurant,Mexican Restaurant,Bakery,Café,Light Rail Station,Bus Station,Thrift / Vintage Store,Theater,Pizza Place,Market
4,Bayview Heights,Park,Breakfast Spot,Latin American Restaurant,Burger Joint,Food Truck,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food


Lastly, I want to get an idea of which neighborhoods, expensive or cheap, have similar venue types. To do this, I will cluster the neighborhoods using *k*-means, separating the data into 4 clusters. This will give me an idea of which cheaper neighborhoods have similar venues to expensive neighborhoods.

In [49]:
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(sf_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [50]:
sorted_venues.insert(0, 'Cluster Labels', kmeans.labels_)

In [51]:
sf_merged = pd.merge(full_df, sorted_venues, on = 'Neighborhood')

sf_merged.head()

,Neighborhood,Average Rent,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Treasure Island,"$2,616",37.823354,-122.370153,0,Baseball Field,Island,Flea Market,Athletics & Sports,Grocery Store,Park,Rugby Pitch,Gym,American Restaurant,Food
1,Van Ness - Civic Center,"$2,944",37.775190,-122.419266,0,Cocktail Bar,Coffee Shop,Café,French Restaurant,Mexican Restaurant,Wine Bar,Sushi Restaurant,New American Restaurant,Boutique,Food & Drink Shop
2,Tenderloin,"$2,944",37.784249,-122.413993,0,Coffee Shop,Theater,Thai Restaurant,Vietnamese Restaurant,Mexican Restaurant,Cocktail Bar,Art Gallery,Bar,Sandwich Place,Burger Joint
3,Downtown District 8 - North East,"$2,956",37.787936,-122.407517,0,Hotel,Boutique,Clothing Store,Coffee Shop,Cocktail Bar,Spa,Jewelry Store,Department Store,Furniture / Home Store,Lounge
4,Marina,"$2,974",37.802984,-122.437472,0,Gym / Fitness Center,Pizza Place,Sushi Restaurant,Mexican Restaurant,Park,Italian Restaurant,Deli / Bodega,Cosmetics Shop,Coffee Shop,Spa


Finally, the I can visualize the clusters by color on the map of San Francisco.

In [52]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12)

x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 1).add_to(map_clusters)
       
map_clusters

The first cluster, Cluster 0, is obviously the largest. It contains neighborhoods with high diversity of venues, many of which are restaurants and other food establishments.

#### Cluster 0

In [53]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[0, 1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,Average Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Treasure Island,"$2,616",Baseball Field,Island,Flea Market,Athletics & Sports,Grocery Store,Park,Rugby Pitch,Gym,American Restaurant,Food
1,Van Ness - Civic Center,"$2,944",Cocktail Bar,Coffee Shop,Café,French Restaurant,Mexican Restaurant,Wine Bar,Sushi Restaurant,New American Restaurant,Boutique,Food & Drink Shop
2,Tenderloin,"$2,944",Coffee Shop,Theater,Thai Restaurant,Vietnamese Restaurant,Mexican Restaurant,Cocktail Bar,Art Gallery,Bar,Sandwich Place,Burger Joint
3,Downtown District 8 - North East,"$2,956",Hotel,Boutique,Clothing Store,Coffee Shop,Cocktail Bar,Spa,Jewelry Store,Department Store,Furniture / Home Store,Lounge
4,Marina,"$2,974",Gym / Fitness Center,Pizza Place,Sushi Restaurant,Mexican Restaurant,Park,Italian Restaurant,Deli / Bodega,Cosmetics Shop,Coffee Shop,Spa
5,Hayes Valley,"$2,983",Clothing Store,Wine Bar,Boutique,Sushi Restaurant,French Restaurant,Pizza Place,Bakery,New American Restaurant,Ice Cream Shop,Concert Hall
6,Russian Hill,"$2,983",Park,Italian Restaurant,Coffee Shop,Sushi Restaurant,Dive Bar,Playground,Pizza Place,Trail,Café,Garden
7,Nob Hill,"$2,996",Italian Restaurant,Hotel,Café,Wine Bar,French Restaurant,Coffee Shop,Bar,Hotel Bar,Gym,Grocery Store
8,Western Addition,"$3,030",Grocery Store,Liquor Store,Jazz Club,Hotel,Bus Stop,Pakistani Restaurant,Café,German Restaurant,Boutique,Museum
9,Bernal Heights,"$3,061",Coffee Shop,Bakery,Italian Restaurant,Mexican Restaurant,Trail,Yoga Studio,Grocery Store,Gourmet Shop,Food Truck,Park


The second cluster, Cluster 1, contains only Lake Shore. The dataframe is a bit deceptive, and contradicts the earlier output of venue frequency. If you scroll back up, you will see that for Lake Shore, the top venue is simply "Lake," as it is here, and the other venues have a frequency of 0. This is because the only venue in Lake Shore *is* the lake. The 2nd-10th most common venues are filler, and so the only real deciding factor in this cluster is the lake.

#### Cluster 1

In [54]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[0, 1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,Average Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Lake Shore,"$3,664",Lake,Yoga Studio,Food Truck,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop


In this third cluster, Cluster 2, it's easy to see that these neighborhoods were grouped together because of their numerous trails and parks, as well as other outdoor venues.

#### Cluster 2

In [55]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[0, 1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,Average Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Golden Gate Heights,"$3,444",Trail,Tennis Court,Playground,Park,Fried Chicken Joint,French Restaurant,Fast Food Restaurant,Field,Furniture / Home Store,Filipino Restaurant
28,Midtown Terrace,"$3,571",Trail,Playground,Monument / Landmark,Yoga Studio,Farmers Market,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop
32,Forest Knolls,"$3,645",Trail,Park,Mexican Restaurant,Chinese Restaurant,Coffee Shop,Mountain,Café,Cafeteria,Salad Place,Food Truck
60,Twin Peaks,"$3,880",Trail,Scenic Lookout,Hill,Garden,Reservoir,Tailor Shop,Bus Stop,Flower Shop,Farmers Market,Fast Food Restaurant
61,John McLaren Park,"$3,892",Park,Dog Run,Pool,Trail,Frozen Yogurt Shop,Fried Chicken Joint,Field,Filipino Restaurant,Fish Market,Flea Market
72,Clarendon Heights,"$4,153",Trail,Playground,Reservoir,Monument / Landmark,Food,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market


This last cluster is similar to the second. Because it only has two venues, the liquor store and the dog run, the rest are filler. In fact, you will note that they are the same filler venues in the same order as the second cluster.

#### Cluster 3

In [56]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[0, 1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,Average Rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Crocker Amazon,"$3,892",Liquor Store,Dog Run,Yoga Studio,Food Truck,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food


## Conclusion/Suggestion

At only 46.27 mi<sup>2</sup>, San Francisco is a small city compared to other famous ones, such as New York, London, Tokyo, etc. Because of this, it is not  surprising that so many neighborhoods ended up in the second cluster. Nearby neighborhoods likely share some venues, making them more homogeneous in the eyes of the clustering algorithm. The benefit of this is that many cheaper neighborhoods have similar amenities to their more expensive counterparts.

The 10 cheapest neighborhoods are as follows:
1. Treasure Island
2. Van Ness - Civic Center
3. Tenderloin
4. Downtown District 8 - Northeast
5. Marina
6. Hayes Valley
7. Russian Hill
8. Nob Hill
9. Western Addition
10. Bernal Heights

As mentioned earlier, however, people may not want to sacrifice entertainment and convenience for lower monthly rent. Although Treasure Island is the cheapest neighborhood, it is lacking in amenities.

Tenderloin, Downtown District 8 - Northeast, Marina, and Hayes Valley are the cheapest neighborhoods that surpass 100 venues. Of these, Marina has the greatest number of unique venues among this group. 

With an average rent below $3,000 per month and a large number of diverse venues, I believe that __Marina__ is the neighborhood best-suited to renters.